# 模型构造

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

## 从区块构建模型

从`tf.keras.Model` 继承 `__init__`和`call`函数。它们分别用于创建模型参数和定义正向传播

In [59]:
# 构建正向传播模型
class MLP(keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = keras.layers.Flatten()
        self.dense1 = keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = keras.layers.Dense(units=10)
    
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

# 一次正向传播
X = tf.random.uniform((2,20))
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.3517177 ,  0.0590828 , -0.09553804,  0.18189082,  0.10262021,
         0.1591081 ,  0.04742083,  0.04850521, -0.0761077 , -0.05269895],
       [-0.57858044,  0.25556022, -0.15854385, -0.09426881,  0.12265214,
         0.30309367,  0.1856965 ,  0.15669292, -0.13691807, -0.14801069]],
      dtype=float32)>

## Sequential

实际上，我们常用的`Sequential`也继承自`tf.keras.Model`。当模型进行正向传播为**简单串联各个层**的计算时，可以通过`Sequential`来更加简单的定义模型

In [110]:
# 直接通过list来构建模型
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(10)
])

model(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.03315611,  0.3800161 ,  0.23285574,  0.02806905, -0.47320196,
         0.35398674, -0.21407685,  0.15584847, -0.27790308,  0.06921495],
       [-0.04699008,  0.2853636 ,  0.03808613, -0.1333533 , -0.4015093 ,
         0.12817937,  0.0796181 ,  0.14587305, -0.09163154,  0.20914048]],
      dtype=float32)>

In [42]:
# 通过add来构建模型
model = keras.Sequential()
model.add(keras.layers.Dense(256, activation=tf.nn.relu))
model.add(keras.layers.Dense(10))

model(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.3122393 ,  0.05919163,  0.03653248, -0.08400317, -0.22641888,
         0.00232723,  0.21166524, -0.07668218,  0.09955931,  0.0278854 ],
       [ 0.21390367, -0.06434882,  0.04908498,  0.01998814, -0.2421408 ,
        -0.21148819,  0.27529937,  0.01524206, -0.09090518,  0.05558806]],
      dtype=float32)>

## 构建复杂模型

虽然`Sequential`可以使构建模型更加简单且不需要定义`call`函数，但是直接继承`tf.keras.Model`类可以极大扩展模型构造的灵活性。下例中，通过`constant`函数创建训练中不被迭代的参数，即常数参数。在正向传播中，除了使用创建的常数参数外，我们还是用了`tensor`的函数和Python控制流，并多次调用相同的层。

In [58]:
class FancyMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(
            tf.random.uniform((20,20)))
        self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        x = self.dense(x)    
        while tf.norm(x) > 1:
            x /= 2
        if tf.norm(x) < 0.8:
            x *= 10
        return tf.reduce_sum(x)

net = FancyMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=4.5308247>

因为`FancyMLP`和`Sequential`都是`tf.keras.Model`的子类，所以可以嵌套调用它们

In [60]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Flatten())
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)

    def call(self, inputs):         
        return self.dense(self.net(inputs))

net = tf.keras.Sequential()
net.add(NestMLP())
net.add(tf.keras.layers.Dense(20))
net.add(FancyMLP())

net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=19.843418>

In [78]:
net.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
nest_mlp (NestMLP)           multiple                  3952      
_________________________________________________________________
dense_72 (Dense)             multiple                  340       
_________________________________________________________________
fancy_mlp_14 (FancyMLP)      multiple                  420       
Total params: 4,712
Trainable params: 4,712
Non-trainable params: 0
_________________________________________________________________


# 模型参数的访问、初始化和共享

In [121]:
net = tf.keras.models.Sequential()
net.add(tf.keras.layers.Flatten())
net.add(tf.keras.layers.Dense(256,activation=tf.nn.relu))
net.add(tf.keras.layers.Dense(10))

X = tf.random.uniform((2,20))
Y = net(X)
Y

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.03510882, -0.34613597, -0.02393222, -0.18034738, -0.05238268,
        -0.08591746,  0.01697223,  0.1875126 ,  0.42745113,  0.31601268],
       [ 0.21769872, -0.36894047, -0.10778795, -0.19691208, -0.07622939,
         0.16868882,  0.0686077 ,  0.12315129,  0.1612009 ,  0.1531213 ]],
      dtype=float32)>

## 访问模型参数

使用`Sequential`构造的神经网络，可以通过`weights`属性来访问网络任一层的权重

In [139]:
# Flatten层无weights和bias被无视
# 一个Dense层会产生两个weights，其中[0]为w, [1]为b
net.weights[0], net.weights[1]

(<tf.Variable 'sequential_54/dense_134/kernel:0' shape=(20, 256) dtype=float32, numpy=
 array([[-0.03383882, -0.11707048,  0.10972074, ...,  0.08872013,
          0.06855111,  0.02397333],
        [ 0.08151212,  0.00056681,  0.03013374, ..., -0.00486116,
          0.03044017,  0.06502134],
        [ 0.132911  ,  0.12222865,  0.09149063, ..., -0.13583285,
          0.13097805, -0.14202355],
        ...,
        [-0.14262432, -0.07902815, -0.00359689, ...,  0.07753824,
         -0.05405951, -0.10552894],
        [-0.09352137, -0.01143943,  0.12965965, ..., -0.12315992,
          0.10722223,  0.05119947],
        [ 0.13653904, -0.08938001,  0.02308467, ..., -0.04827231,
          0.11051106,  0.05444454]], dtype=float32)>,
 <tf.Variable 'sequential_54/dense_134/bias:0' shape=(256,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 

## 初始化参数

模型的默认初始化方法为: 权重参数元素为[-0.07, 0.07]之间均匀分布的随机数, 偏差则为0。通常我们需要使用其他方法来初始化权重，在下面的例子中，将权重参数初始化为均值为0，标准差为0.01的正态分布随机数，偏差为0/1

In [140]:
class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(
            units=10,
            activation=None,
            kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),  # weight
            bias_initializer=tf.zeros_initializer()   # bias
        )
        self.d2 = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
            bias_initializer=tf.ones_initializer()
        )

    def call(self, input):
        output = self.d1(input)
        output = self.d2(output)
        return output

net = Linear()
net(X)
net.get_weights()

[array([[ 0.00909971,  0.00596697, -0.00332779,  0.0149344 , -0.00717133,
         -0.0014989 , -0.0087957 ,  0.00282516, -0.00195604, -0.00067179],
        [ 0.00490627, -0.01633284,  0.00755273,  0.00136864, -0.02045935,
         -0.01924302, -0.01074835, -0.00238334, -0.00282798, -0.01072354],
        [-0.00421563, -0.0090111 ,  0.02130167, -0.01302595,  0.00599386,
         -0.01292618,  0.01114051, -0.01639823,  0.00310355,  0.01305074],
        [ 0.00324933,  0.01948626, -0.02190285,  0.00387663, -0.01059838,
          0.02322861,  0.01128075,  0.02057246,  0.00265373, -0.01068667],
        [-0.00877597, -0.00808634,  0.00227806, -0.00323111, -0.00474815,
         -0.02480922,  0.00621112, -0.0022819 , -0.00989668, -0.01789486],
        [ 0.00881773, -0.00289748, -0.004485  ,  0.00593094, -0.01509368,
          0.00144251,  0.00610703,  0.02330093,  0.00999746, -0.00072805],
        [ 0.00837816,  0.0138937 , -0.00884771, -0.00474632, -0.00267756,
          0.00205341, -0.0243916

可以使用`tf.keras.initializers` 来自定义初始化

In [141]:
def my_init():
    return tf.keras.initializers.Ones()

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, kernel_initializer=my_init()))

Y = model(X)
model.weights[0]

<tf.Variable 'sequential_55/dense_138/kernel:0' shape=(20, 64) dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>

## 延后初始化

我们在使用`Sequential`里的`layers`进行创建全连接层的时候，都没有指定输入的个数，由于输入的个数未知，因此该层的w和b也没有办法进行初始化，只有在数据传入网络之后，才会初始化其各层的w和b，这叫做延后初始化

In [5]:
net = tf.keras.models.Sequential()
net.add(tf.keras.layers.Flatten())
net.add(tf.keras.layers.Dense(256,activation=tf.nn.relu))
net.add(tf.keras.layers.Dense(10))

In [7]:
# 第一层网络的w还没有初始化，因此报错

try:
    net.weights[0]
except:
    print('The w of first layer has not been created')

The w of first layer has not been created


In [10]:
# 在传入数据后，每一层都有了参数

X = tf.random.uniform((2,20))
Y = net(X)

net.weights[0]

<tf.Variable 'sequential_1/dense_2/kernel:0' shape=(20, 256) dtype=float32, numpy=
array([[-0.07148068, -0.1116434 ,  0.13821396, ...,  0.14516154,
        -0.0500281 ,  0.12749329],
       [ 0.05667473, -0.07406509,  0.03059828, ...,  0.02128822,
         0.07427762,  0.13693061],
       [ 0.05781633,  0.03457713, -0.0407213 , ...,  0.00135897,
        -0.04260268, -0.06990696],
       ...,
       [-0.12363131,  0.07524391, -0.04777505, ...,  0.11447403,
        -0.12888934, -0.08623327],
       [ 0.09619942,  0.08833931, -0.0370017 , ..., -0.07345831,
         0.00284949,  0.05666068],
       [ 0.13059112,  0.12397876, -0.13316517, ...,  0.03502269,
        -0.14035225, -0.1291043 ]], dtype=float32)>

# 自定义层

`tf.keras`虽然提供了大量常用的层，但是有时候我们仍希望自定义层。

## 自定义无参数的层

下例中，通过继承`tf.keras.layers.Layer`自定义了一个将输入减掉均值后输出的层。并将该层的计算定义在了`call`函数里面。这个层里面不含参数。

In [14]:
class CenteredLayer(tf.keras.layers.Layer):
    def __inif__(self):
        super().__init__()
    
    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [15]:
net = tf.keras.models.Sequential()
net.add(tf.keras.layers.Flatten())
net.add(tf.keras.layers.Dense(20))
net.add(CenteredLayer())

Y = net(X)
Y

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[-0.20958811,  0.20888385,  0.8755732 , -0.29745674, -0.02969303,
         0.10262606,  1.1162755 ,  0.3695641 ,  0.31335768, -0.70059276,
        -0.14199486, -0.16570246,  0.03128872, -0.10945373, -0.6193166 ,
        -0.24931586,  0.1426819 ,  0.14857855,  0.06305855,  0.75794154],
       [-0.42829385,  0.2524372 ,  1.3564847 , -0.21308675, -0.4226271 ,
        -0.11136381,  0.35428724, -0.02224728,  0.9663147 , -0.3997038 ,
        -0.38516703, -0.58401126, -0.45115656, -0.71701694, -0.9077915 ,
        -0.24290758,  0.42526743, -0.12259867, -0.5712693 ,  0.6177351 ]],
      dtype=float32)>

## 自定义有参数的层

build一般用来往Layer中添加变量

In [22]:
class myDense(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(name='w',
                                 shape=[input_shape[-1], self.units],
                                 initializer=tf.random_normal_initializer())
        self.b = self.add_weight(name='b',
                                 shape=[1, self.units], 
                                 initializer=tf.zeros_initializer())
    
    def call(self, inputs):
        y_pred = tf.matmul(inputs, self.w) + self.b
        return y_pred

In [28]:
net = tf.keras.models.Sequential()
net.add(myDense(8))
net.add(myDense(1))

net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.00166104],
       [-0.01320496]], dtype=float32)>

# 读取和储存

## 读取储存NDarray

构建了一个`tensor`, 使用`np.save`进行保存，然后使用`np.load`读取

In [48]:
x = tf.ones(3)
np.save('files/class5_4.1 - x.npy', x)

x2 = np.load('files/class5_4.1 - x.npy')
x2

array([1., 1., 1.], dtype=float32)

也可以储存一列`tensor`， 然后再将其读回内存

In [49]:
y = tf.zeros(4)
np.save('files/class5_4.1 - xy.npy', [x,y])

x2, y2 = np.load('files/class5_4.1 - xy.npy', allow_pickle=True)
(x2, y2)

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

也可以储存并读取一个从字符串映射到`tensor`的字典

In [50]:
mydict = {'x':x, 'y':y}
np.save('files/class5_4.1 - mydict.npy', mydict)

mydict2 = np.load('files/class5_4.1 - mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

## 读取储存模型参数

新建一个模型并进行初始化参数，然后使用`save_weights`将其储存

In [54]:
X = tf.random.normal((2,20))
net1 = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(10)
])
Y1 = net1(X)
Y1

net1.save_weights('files/class5_4.2 - saved_model.h5')

构建一个与net1一致的NN，将其初始化之后，使用`load_weights`读取net1的权重参数，然后进行一次正向传播。Y1与Y2的值一致，表示net1与net2的权重参数一致。

In [56]:
net2 = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(10)
])
net2(X)
net2.load_weights('files/class5_4.2 - saved_model.h5')
Y2 = net2(X)
Y2 == Y1

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

# GPU计算

因该部分需要GPU，而MAC环境只有CPU。该部分内容略去，若要理解则自行去书4.6看代码

# 重点总结

`tf.keras.layers.Layer`与`tf.keras.Model`的类同与区别
- `tf.keras.Model`可以使用`.fit()`/`.evaluate()`/`.predict()`等。 而在`tf.keras.layers.Layer`不可以使用上述方法
- 一般而言，在构建层/块的时候, 倾向于使用`tf.keras.layers.Layer` (如Inception Block /ResNet Block /Convolutional Layer 等)
- 一般而言，在构建网络的时候, 倾向于使用`tf.keras.Model` (如自己构建的DL_Model/NN等)
- 他们两个其实内在的参数使用是非常类似的

在`tf.keras.layers.Layer`与`tf.keras.Model`中, 使用`super(class_name, self).__init__()`来继承上述类的属性和方法
- `__init__`中, 里面含有从类中继承来的属性, 并且我们可以在这里面定义自己类的对象
- `build`中, 用于定义 该层/网络 中额外的 参数(包括超参数/固定参数等)
- `call`中, 用于定义该层/网络中的正向传播逻辑 

例子: class7_5.2/ class8_1.2 等